In [33]:
import numpy as np
import random
from datetime import datetime, timedelta, time
import math
from collections import defaultdict
import pandas as pd
from sqlalchemy import create_engine,text
import matplotlib.pyplot as plt

In [34]:
    
class Ruang:
    def __init__(self, nama, tipe_ruang):
        self.nama = nama
        self.tipe_ruang = tipe_ruang
        self.jadwal = defaultdict(lambda: defaultdict(lambda: None))
    
    def __repr__(self):
        return f"Ruang(nama={self.nama}, tipe_ruang={self.tipe_ruang})"

In [35]:
    
class Dosen:
    def __init__(self, nama):
        self.nama = nama
        self.jadwal = defaultdict(lambda: defaultdict(lambda: None))

    
    def __repr__(self):
        return f"Dosen(nama={self.nama})"

In [36]:
class Matakuliah:
    def __init__(self, matkul, dosen, sks, kelas, status, id_perkuliahan, id_semester,semester,kategori,prodi): #tambah id_perkuliahan
        
        self.id_perkuliahan = id_perkuliahan
        self.matkul = matkul
        self.dosen = dosen
        self.sks = sks
        self.status = status
        self.kelas = kelas
        self.id_semester = id_semester
        self.semester = semester
        self.kategori = kategori
        self.prodi=prodi
        self.ruang_needed = self.set_ruang(kategori)

    def __repr__(self):
        return (f"matkul(matkul={self.matkul}, dosen={self.dosen}, sks={self.sks}, status={self.status})")
    
    def set_ruang(self, kategori):
        if kategori == "Teori":
            return ["Kelas"]
        elif kategori == "Praktikum":
            return ["Lab"]
        elif kategori == "Gabungan":
            return ["Kelas", "Lab"]
        else:
            return []

In [ ]:
class PenjadwalanSA:
    def __init__(self, initial_temperature=10000, cooling_rate=0.9999, max_iterations=200000):
        self.initial_temperature = initial_temperature
        self.cooling_rate = cooling_rate
        self.max_iterations = max_iterations
        
        # Data penjadwalan
        self.daftar_dosen = []
        self.daftar_ruang = []
        self.daftar_matkul = []
        self.daftar_hari = ['Senin', 'Selasa', 'Rabu', 'Kamis', 'Jumat']
        

        # Slot waktu
        self.durasi_slot = timedelta(minutes=50)
        self.jam_mulai = datetime.strptime("07:00", "%H:%M")
        self.jam_selesai = datetime.strptime("19:05", "%H:%M")
        self.slot_istirahat = [
            (datetime.strptime("09:30", "%H:%M"), datetime.strptime("09:45", "%H:%M")),
            (datetime.strptime("12:15", "%H:%M"), datetime.strptime("12:45", "%H:%M")),
            (datetime.strptime("15:15", "%H:%M"), datetime.strptime("15:30", "%H:%M")),            
            (datetime.strptime("18:00", "%H:%M"), datetime.strptime("18:15", "%H:%M")),
        ]
        self.daftar_slot = self.generate_slot_waktu()
        print(len(self.daftar_slot))
        self.prodi_jadwal = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: None))))

        self.baca_datamk()
        self.baca_dataruang()

    def baca_datamk(self):
        # baca dari database tabel perkuliahan dengan id generate 1 --belum
        db = 'mysql+pymysql://root:@localhost/db_optimasi1'
        engine = create_engine(db)
        query = text("""
            SELECT tb_rombel.nama_kelas AS kelas,
                tb_matakuliah.nama_matakuliah AS matakuliah,
                tb_dosen.nama_dosen AS dosen,
                tb_matakuliah.sks AS sks,
                tb_matakuliah.status AS status,
                tb_matakuliah.kategori AS kategori,
                id_perkuliahan,
                id_semester,
                tb_matakuliah.nama_semester AS semester,
                tb_prodi.nama_prodi AS prodi
            FROM tb_perkuliahan
            JOIN tb_matakuliah ON tb_perkuliahan.kode_matakuliah = tb_matakuliah.kode_matakuliah
            JOIN tb_rombel ON tb_perkuliahan.id_kelasrombel = tb_rombel.id_kelasrombel
            JOIN tb_dosen ON tb_perkuliahan.kode_dosen = tb_dosen.kode_dosen
            JOIN tb_generate ON tb_generate.id_generate = tb_rombel.id_generate
            JOIN tb_prodi ON tb_perkuliahan.kode_prodi = tb_prodi.kode_prodi
            AND tb_generate.id_generate = tb_matakuliah.id_generate 
            AND tb_generate.id_generate = tb_dosen.id_generate
            AND tb_generate.id_generate = tb_prodi.id_generate
            AND tb_generate.id_generate = 'G0706-02'
            
        """)
        with engine.connect() as connection:
            df_matkul = pd.read_sql_query(query, connection)

        # Iterasi per baris untuk menambahkan data
        for _, row in df_matkul.iterrows():
            dosen_obj = self.tambah_dosen(row['dosen'])
            self.tambah_matkul(row['matakuliah'], dosen_obj, row['sks'], row['kelas'], row['status'], row['id_perkuliahan'],row['id_semester'],row['semester'],row['kategori'],row['prodi'])
        
        
    def baca_datadosen(self):
        # baca dari database tabel perkuliahan dengan id generate 1 --belum
        db = 'mysql+pymysql://root:@localhost/db_optimasi1'
        engine = create_engine(db)
        query = text("""
            SELECT nama_dosen AS dosen
            FROM tb_dosen
            JOIN tb_generate ON tb_dosen.id_generate = tb_generate.id_generate
            AND tb_generate.id_generate = 'G0706-02'
        """)
        with engine.connect() as connection:
            df_dosen = pd.read_sql_query(query, connection)
        for _, row in df_dosen.iterrows():
            self.tambah_dosen(row['dosen'])

    def baca_dataruang(self):
        # berasal dari database dengan id generate 1
        # Iterasi per baris untuk menambahkan data
        db = 'mysql+pymysql://root:@localhost/db_optimasi1'
        engine = create_engine(db)
        query = text("""
        SELECT nama_ruangan, 
            status_ruangan 
        FROM tb_ruang 
        JOIN tb_generate ON tb_ruang.id_generate = tb_generate.id_generate
        AND tb_generate.id_generate = 'G0706-02'
        
        """)
        with engine.connect() as connection:
            df_ruang = pd.read_sql_query(query, connection)

        # Iterasi per baris untuk menambahkan data
        for _, row in df_ruang.iterrows():
            self.tambah_ruang(row['nama_ruangan'], row['status_ruangan'])
    
    def tambah_ruang(self, nama, tiperuang):
        ruang = Ruang(nama,tiperuang)
        self.daftar_ruang.append(ruang)
        return ruang
    
    def tambah_dosen(self, nama):
        for d in self.daftar_dosen:
            if d.nama == nama:
                return d
        dosen = Dosen(nama)
        self.daftar_dosen.append(dosen)
        return dosen

    
    def tambah_matkul(self,matkul, dosen, sks, kelas, status,id_perkuliahan,id_semester,semester,prodi,kategori):
        matkul = Matakuliah(matkul, dosen, sks, kelas, status,id_perkuliahan,id_semester,semester,prodi,kategori) #tambahkan id_perkuliahan
        self.daftar_matkul.append(matkul)
        return matkul
        
    def generate_slot_waktu(self):
        daftar_slot = []
        waktu_mulai = self.jam_mulai

        while waktu_mulai + self.durasi_slot <= self.jam_selesai:
            waktu_selesai = waktu_mulai + self.durasi_slot

            konflik_istirahat = False
            for istirahat_mulai, istirahat_selesai in self.slot_istirahat:
                if not (waktu_selesai <= istirahat_mulai or waktu_mulai >= istirahat_selesai):
                    # Slot tumpang tindih dengan istirahat
                    waktu_mulai = istirahat_selesai  # lanjut setelah istirahat
                    konflik_istirahat = True
                    break

            if not konflik_istirahat:
                daftar_slot.append((waktu_mulai.strftime("%H:%M")))
                waktu_mulai = waktu_selesai
        return daftar_slot
    
    def hitung_sks(self, sks, kategori):
        if kategori == "Teori":
            return sks 
        elif kategori == "Praktikum":
            return sks * 3
        elif kategori == "Gabungan":
            return 7 
        return  0
    
    def jam_sks(self, sks, kategori):
        total_sks = self.hitung_sks(sks,kategori)
        if total_sks <= 0:
            print(f" SKS tidak valid: {sks}, kategori: {kategori}")
        return math.ceil(total_sks * 50 / self.durasi_slot.total_seconds() * 60)
    
    def get_ruang_valid(self,matkul):
        ruang_valid = [ruang for ruang in self.daftar_ruang if not matkul.ruang_needed or any(t in ruang.tipe_ruang for t in matkul.ruang_needed)]
        return random.choice(ruang_valid) if ruang_valid else random.choice(self.daftar_ruang)
    
    def simulated_annealing(self):
        current_solution = self.solusi_awal()
        current_energy = self.calculate_energy(current_solution)
        best_solution = current_solution
        best_energy = current_energy
        temperature = self.initial_temperature

        for _ in range(self.max_iterations):
            neighbor_solution = self.get_neighbor(current_solution)
            neighbor_energy = self.calculate_energy(neighbor_solution)

            if self.accept_probability(current_energy, neighbor_energy, temperature) > random.random():
                current_solution = neighbor_solution
                current_energy = neighbor_energy

            if current_energy < best_energy:
                best_solution = current_solution
                best_energy = current_energy

            temperature *= self.cooling_rate
            if _ % 10000 == 0:
                print(f"Iterasi {_} | Skor saat ini: {current_energy} | Skor terbaik: {best_energy} | Temperatur: {temperature:.4f}")

        self.apply_solution(best_solution)
        self.best_solution = best_solution 
        
    def solusi_awal(self):
        solution = []
        for matkul in self.daftar_matkul:
            ruang = self.get_ruang_valid(matkul)
            hari = random.choice(self.daftar_hari)
            durasi = self.jam_sks(matkul.sks, matkul.kategori)
            max_jam_mulai = len(self.daftar_slot) - durasi

            if max_jam_mulai <= 0:
                print(f"⚠️ Matkul {matkul.nama} ({durasi} slot) tidak muat dalam hari (slot tersedia: {len(self.daftar_slot)})")
                continue  # Skip matkul yang tidak muat

            jam_mulai = random.randint(0, max_jam_mulai)
            solution.append((matkul, ruang, hari, jam_mulai))
        return solution
    
    def get_neighbor(self, solution):
        neighbor = solution.copy()
        index = random.randint(0, len(neighbor) - 1)
        matkul, _, _, _ = neighbor[index]
        ruang = self.get_ruang_valid(matkul)
        hari = random.choice(self.daftar_hari)
        jam_mulai = random.randint(0, len(self.daftar_slot) - self.jam_sks(matkul.sks,matkul.kategori))
        neighbor[index] = (matkul, ruang, hari, jam_mulai)
        return neighbor
    
    def calculate_energy(self, solution):
        conflicts = 0
        for i, (mk1, r1, h1, j1) in enumerate(solution):
            slots_needed1 = self.jam_sks(mk1.sks,mk1.kategori)
            if mk1.ruang_needed and not any(t in r1.tipe_ruang for t in mk1.ruang_needed):
                conflicts += 10  # Penalti besar untuk matakuliah yang tidak di ruang yang sesuai
            for mk2, r2, h2, j2 in solution[i+1:]:
                slots_needed2 = self.jam_sks(mk2.sks,mk2.kategori)
                if h1 == h2:
                    if r1 == r2 and max(j1, j2) < min(j1 + slots_needed1, j2 + slots_needed2):
                        conflicts += 1
                    if mk1.dosen == mk2.dosen and max(j1, j2) < min(j1 + slots_needed1, j2 + slots_needed2):
                        conflicts += 1
                    if mk1.prodi == mk2.prodi and mk1.semester == mk2.semester and max(j1, j2) < min(j1 + slots_needed1, j2 + slots_needed2):
                        conflicts += 1
        return conflicts
    
    def accept_probability(self, current_energy, neighbor_energy, temperature):
        if neighbor_energy < current_energy:
            return 1.0
        return math.exp((current_energy - neighbor_energy) / temperature)
    
    def apply_solution(self, solution):
        self.reset_jadwal()
        for matkul, ruang, hari, jam_mulai in solution:
            slots_needed = self.jam_sks(matkul.sks,matkul.kategori)
            for i in range(slots_needed):
                if jam_mulai + i < len(self.daftar_slot):  # pastikan jam_list
                    ruang.jadwal[hari][jam_mulai + i] = matkul
                    matkul.dosen.jadwal[hari][jam_mulai + i] = matkul
                    self.prodi_jadwal[matkul.prodi][matkul.semester][hari][jam_mulai + i] = matkul

    def reset_jadwal(self):
        for ruang in self.daftar_ruang:
            ruang.jadwal = defaultdict(lambda: defaultdict(lambda: None))
        for dosen in self.daftar_dosen:
            dosen.jadwal = defaultdict(lambda: defaultdict(lambda: None))
        self.prodi_jadwal = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: None))))
    
    def generate_jadwal(self):
        self.simulated_annealing()
        return self.best_solution

    
    def cari_ruang(self, matakuliah, hari, jam):
        for ruang in self.daftar_ruang:
            if ruang.jadwal[hari][jam] == matakuliah:
                return ruang
        return None
    def tampilkan_jadwal(self):
        print("\n========== JADWAL PERKULIAHAN ==========\n")
        for prodi in self.prodi_jadwal:
            print(f"\n📚 PRODI: {prodi}")
            for semester in sorted(self.prodi_jadwal[prodi]):
                print(f"\n  🎓 Semester: {semester}")
                for hari in self.daftar_hari:
                    print(f"\n    📅 Hari: {hari}")
                    for jam_index in sorted(self.prodi_jadwal[prodi][semester][hari]):
                        matkul = self.prodi_jadwal[prodi][semester][hari][jam_index]
                        if matkul:
                            ruang = self.cari_ruang(matkul, hari, jam_index)
                            jam = self.daftar_slot[jam_index]
                            print(f"      🕒 {jam} - {matkul.matkul} | {matkul.dosen} | {ruang.nama} | Kelas: {matkul.kelas}")

    def df_hasiljadwal(self, solution):
        data=[]
        for matkul, ruang, hari, jam_mulai in solution:
            butuh_slot = self.jam_sks(matkul.sks, matkul.kategori)
            jam_selesai = jam_mulai + butuh_slot - 1
            waktu_mulai = self.daftar_slot[jam_mulai]              
            waktu_selesai = self.daftar_slot[jam_selesai]          

            data.append({
                'id_perkuliahan': matkul.id_perkuliahan,
                'hari': hari,
                'jam_mulai': waktu_mulai,
                'jam_selesai': waktu_selesai,
                'kelas': matkul.kelas,
                'mata_kuliah': matkul.matkul,
                'nama_dosen': matkul.dosen.nama,
                'ruang': ruang.nama if hasattr(ruang, 'nama') else ruang,
                'semester': matkul.semester
            })
        df = pd.DataFrame(data)
        return df
    
    def simpan_optimasi(self, df, table_name='tb_hasil'):
        db = 'mysql+pymysql://root:@localhost/db_optimasi1'
        engine = create_engine(db)
        df.to_sql(table_name, con=engine, if_exists='append', index=False)
        print(f'Data berhasil disimpan ke database {table_name}')
        with engine.begin() as conn:
            query = text("UPDATE tb_generate SET status = :status WHERE id_generate = 'G0706-02'")
            conn.execute(query, {"status": "sudah", "id_generate": "G0706-02"})
    

    
sa = PenjadwalanSA()

hasil = sa.generate_jadwal()
sa.tampilkan_jadwal()
df_jadwal = sa.df_hasiljadwal(hasil)
print(df_jadwal)
sa.simpan_optimasi(df_jadwal)




13
Iterasi 0 | Skor saat ini: 219 | Skor terbaik: 219 | Temperatur: 9999.0000
Iterasi 10000 | Skor saat ini: 209 | Skor terbaik: 165 | Temperatur: 3678.2426
Iterasi 20000 | Skor saat ini: 200 | Skor terbaik: 161 | Temperatur: 1353.0822
Iterasi 30000 | Skor saat ini: 193 | Skor terbaik: 153 | Temperatur: 497.7462
Iterasi 40000 | Skor saat ini: 176 | Skor terbaik: 153 | Temperatur: 183.1014
Iterasi 50000 | Skor saat ini: 216 | Skor terbaik: 153 | Temperatur: 67.3559
Iterasi 60000 | Skor saat ini: 169 | Skor terbaik: 153 | Temperatur: 24.7776
Iterasi 70000 | Skor saat ini: 172 | Skor terbaik: 153 | Temperatur: 9.1147
Iterasi 80000 | Skor saat ini: 153 | Skor terbaik: 137 | Temperatur: 3.3529
Iterasi 90000 | Skor saat ini: 125 | Skor terbaik: 104 | Temperatur: 1.2334
Iterasi 100000 | Skor saat ini: 78 | Skor terbaik: 72 | Temperatur: 0.4537
Iterasi 110000 | Skor saat ini: 46 | Skor terbaik: 46 | Temperatur: 0.1669
Iterasi 120000 | Skor saat ini: 42 | Skor terbaik: 42 | Temperatur: 0.0614
I

AttributeError: 'PenjadwalanSA' object has no attribute 'id_generate'